In [1]:
import joblib
import json
import os
import pandas as pd

from flask import Flask
from flask import jsonify
from flask import request
# from flask_restful import Api, Resource
import sys

os.chdir('/home/jupyter/tfg-sb-meal-delivery-prediction/')

from src.data.data_collect import read_train_data
from src.model.xgboost_model import preprocess_data, train_xgboost_model

In [2]:
df_train = read_train_data()

In [3]:
with open('./src/test/test_api_train.json') as f:
    config = json.load(f)

In [4]:
config

{'center_id': 55, 'meal_id': 1993}

In [5]:
# Preprocess the dataframe
center_id = config['center_id']
meal_id = config['meal_id']

In [6]:
df_preprocessed = preprocess_data(df_train, center_id, meal_id)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['day'] = df_processed.date.dt.day
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [7]:
features = list(df_preprocessed.drop(columns='num_orders').columns)

In [8]:
regressor_model, rmse = train_xgboost_model(df_preprocessed)

RMSE: 45.64661830504204


In [9]:
result = {
    'features' : features,
    'rmse' : rmse
}

In [10]:
result

{'features': ['week',
  'checkout_price',
  'base_price',
  'emailer_for_promotion',
  'homepage_featured',
  'city_code',
  'region_code',
  'op_area',
  'day',
  'month',
  'year',
  'quarter'],
 'rmse': 45.64661830504204}

# Test predict API

In [11]:
from src.data.data_collect import read_test_data

In [12]:
df_test = read_test_data()

In [13]:
df_test_processed = preprocess_data(df_test, center_id, meal_id)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['day'] = df_processed.date.dt.day
/home/jupyter/tfg-sb-meal-delivery-prediction/src/model/xgboost_model.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [24]:
import requests

In [26]:
API_ENDPOINT = 'http://localhost:5000/test'

In [29]:
r = requests.get(url = API_ENDPOINT)

In [32]:
r.json()

{'response': 'test successful!!!!!'}

In [163]:
row = json.loads(df_test_processed.head(5).reset_index().to_json(orient='records'))

In [164]:
row

[{'date': 1483228800000,
  'week': 146,
  'checkout_price': 160.11,
  'base_price': 159.11,
  'emailer_for_promotion': 0,
  'homepage_featured': 0,
  'city_code': 647,
  'region_code': 56,
  'op_area': 2.0,
  'day': 1,
  'month': 1,
  'year': 2017,
  'quarter': 1},
 {'date': 1483833600000,
  'week': 147,
  'checkout_price': 156.2,
  'base_price': 156.2,
  'emailer_for_promotion': 0,
  'homepage_featured': 0,
  'city_code': 647,
  'region_code': 56,
  'op_area': 2.0,
  'day': 8,
  'month': 1,
  'year': 2017,
  'quarter': 1},
 {'date': 1484438400000,
  'week': 148,
  'checkout_price': 158.14,
  'base_price': 159.14,
  'emailer_for_promotion': 0,
  'homepage_featured': 0,
  'city_code': 647,
  'region_code': 56,
  'op_area': 2.0,
  'day': 15,
  'month': 1,
  'year': 2017,
  'quarter': 1},
 {'date': 1485043200000,
  'week': 149,
  'checkout_price': 158.11,
  'base_price': 159.11,
  'emailer_for_promotion': 0,
  'homepage_featured': 0,
  'city_code': 647,
  'region_code': 56,
  'op_area': 2

In [165]:
API_ENDPOINT = 'http://localhost:5000/predict'

In [187]:
r = requests.post(url = API_ENDPOINT, json=row)

In [188]:
r

<Response [200]>

In [189]:
r.json()

{'0': '4.912183',
 '1': '4.881111',
 '2': '4.8266926',
 '3': '4.8266926',
 '4': '4.799608'}

# Useful links
+ [Curl rest API](https://linuxize.com/post/curl-rest-api/)
+ [Python-How to send a dataframe to external API (push)](https://stackoverflow.com/questions/63035336/python-how-to-send-a-dataframe-to-external-api-push)
+ [Python Requests post() Method](https://www.w3schools.com/python/ref_requests_post.asp)
+ [Return two reponses in Flask/JavaScript](https://stackoverflow.com/questions/39460769/return-two-responses-in-flask-javascript)